In [ ]:
!pip install streamlit --break-system-packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.5 MB/s eta 0:00:00


In [ ]:
!pip install -U plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:
pip install --ignore-installed blinker


In [ ]:
import pandas as pd
import streamlit as st
import matplotlib.pyplot as plt

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

st.title(" Data Visualization ")

# 1) رفع الملف
file = st.file_uploader("ارفع ملف CSV", type=["csv"], key="up")
if not file:
    st.info("ارفع ملف من نوع csv")
    st.stop()


# 2) قراءة ومعاينة سريعة
df = pd.read_csv(file)
#st.dataframe(df.head(100), use_container_width=True)
#choose the number of row
n_row=st.slider("اختار عدد الصفوف لعرضها",min_value=5,max_value=len(df))
n_col=st.multiselect("اختار الأعمدة",df.columns.tolist())
st.write(df.head(n_row)[n_col])
st.write(" عدد الصفوف والاعمدة",df.shape)
# 3) تجهيز قوائم الأعمدة
num_cols = df.select_dtypes(include=np.number).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]

# 4) اختيار نوع الرسم (واحد فقط لتبسيط الكود)
plot = st.selectbox("اختر نوع الرسم", ["Histogram", "Scatter", "Bar count", "Box","Pie","Heatmap"], key="plot")

# === Histogram ===
if plot == "Histogram":
    if not num_cols:
        st.warning("لا توجد أعمدة رقمية.")
    else:
        col = st.selectbox("العمود الرقمي", num_cols, key="his_col")
        bins = st.slider("عدد الصناديق", 5, 80, 30, key="his_bins")
        fig = px.histogram(df, x=col, nbins=bins, template="plotly_white")
        st.plotly_chart(fig, use_container_width=True)

# === Scatter ===
elif plot == "Scatter":
    if len(num_cols) < 2:
        st.warning("نحتاج عمودين رقميين على الأقل.")
    else:
        x = st.selectbox("المحور X", num_cols, key="sc_x")
        y = st.selectbox("المحور Y", [c for c in num_cols if c != x], key="sc_y")
        color = st.selectbox("لون (اختياري)", ["بدون"] + df.columns.tolist(), key="sc_color")
        color = None if color == "بدون" else color
        fig = px.scatter(df, x=x, y=y, color=color, opacity=0.7, template="plotly_white")
        st.plotly_chart(fig, use_container_width=True)

# === Bar count ===
elif plot == "Bar count":
    if not cat_cols:
        st.warning("لا توجد أعمدة فئوية مناسبة.")
    else:
        c = st.selectbox("عمود فئوي", cat_cols, key="bar_cat")
        counts = df[c].value_counts(dropna=False).reset_index()
        counts.columns = [c, "count"]
        fig = px.bar(counts, x=c, y="count", text_auto=True, template="plotly_white")
        fig.update_xaxes(type="category")
        st.plotly_chart(fig, use_container_width=True)

# === Box ===
elif plot == "Box":
    if not num_cols:
        st.warning("لا توجد أعمدة رقمية.")
    else:
        y = st.selectbox("العمود الرقمي", num_cols, key="box_y")
        x = st.selectbox("تجميع حسب (اختياري)", ["بدون"] + cat_cols, key="box_x")
        x = None if x == "بدون" else x
        fig = px.box(df, x=x, y=y, template="plotly_white", points="outliers")
        st.plotly_chart(fig, use_container_width=True)
# === Pie (counts) ===
elif plot == "Pie":
    # لو ما في أعمدة فئوية، نسمح بأي عمود ونعدّه فئوي
    choices = cat_cols if cat_cols else df.columns.tolist()
    c = st.selectbox("عمود فئوي", choices, key="pie_cat")
    topn = st.slider("أعلى  فئات", 3, 30, 8, key="pie_topn")
    hole = st.slider("حجم  (Donut)", 0.0, 0.6, 0.25, 0.05, key="pie_hole")
    counts = df[c].value_counts(dropna=False).reset_index()
    counts.columns = [c, "count"]
    fig = px.pie(counts.head(topn), values="count", names=c, hole=hole, template="plotly_white")
    st.plotly_chart(fig, use_container_width=True)

# --- Heatmap ---
elif plot=="Heatmap":
    st.write("خريطة حرارية لمعاملات الارتباط بين الأعمدة الرقمية")
    num_cols = df.select_dtypes(include=np.number).columns.tolist()
    if len(num_cols) < 2:
        st.warning("لا توجد أعمدة رقمية كافية لرسم Heatmap (مطلوب عمودان رقميان على الأقل).")
    else:
        sel = st.multiselect("اختر الأعمدة الرقمية", num_cols, default=num_cols, key="hm_cols")
        method = st.selectbox("طريقة الارتباط", ["pearson", "spearman", "kendall"], key="hm_method")

        if len(sel) >= 2:
            corr = df[sel].corr(method=method).round(2)
            fig = px.imshow(
                corr, text_auto=True, aspect="auto",
                color_continuous_scale="RdBu_r",
                origin="lower", template="plotly_white", height=500
            )
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("اختر عمودين رقميين على الأقل لعرض الخريطة الحرارية.")



Overwriting app.py


In [ ]:
!pip install -q streamlit pyngrok


In [ ]:
from pyngrok import ngrok
import subprocess, time, os, signal

# (اختياري) أوقف أي تشغيل سابق
try:
    # kill streamlit on 8501 if running
    import psutil, socket
except:
    !pip install -q psutil
import psutil

for p in psutil.process_iter(attrs=["pid","name","cmdline"]):
    try:
        if p.info["cmdline"] and "streamlit" in " ".join(p.info["cmdline"]):
            p.kill()
    except Exception:
        pass

# ضَعْ التوكن الصحيح هنا فقط (قيمته من لوحة ngrok)
NGROK_TOKEN = "33YyvvMZI7XkokUepoxECAltdgv_3FFpr2Zay1vzw5uCxSD4T"
ngrok.set_auth_token(NGROK_TOKEN)

# شغّل ستريملِت
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

# افتح نفق ngrok
tunnel = ngrok.connect(8501)
print("🔗 افتح التطبيق هنا:", tunnel.public_url)


🔗 افتح التطبيق هنا: https://untrustworthy-splurgy-cannon.ngrok-free.dev
